In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def setparam(row,column,lr,r,n):
    x = row
    y = column
    learning_rate = float(lr)
    radius = float(r)
    num_iter = n
    return x,y,learning_rate,radius,num_iter

In [3]:
def generate_index_matrix(x,y):
        return tf.constant(np.array(list(iterator(x, y))))
    
def iterator(x, y):
    for i in range(x):
        for j in range(y):
            yield np.array([i, j])

In [4]:
x,y,learning_rate,radius,num_iter = setparam(6,6,0.5,0.5,100)
input_dim = 4

In [5]:
#Initializing variables and placeholders
weights = tf.Variable(tf.random_normal([x*y, input_dim]))
locations = generate_index_matrix(x, y)
ip = tf.placeholder("float", [input_dim])
iter_ip = tf.placeholder("float")

In [6]:
#Calculating BMU
input_matix = tf.stack([ip for i in range(x*y)])
distances = tf.sqrt(tf.reduce_sum(tf.pow(tf.subtract(weights, input_matix), 2), 1))
bmu = tf.argmin(distances, 0)

#Get BMU location
mask = tf.pad(tf.reshape(bmu, [1]), np.array([[0, 1]]))
size = tf.cast(tf.constant(np.array([1, 2])), dtype=tf.int64)
bmu_location = tf.reshape(tf.slice(locations, mask, size), [2])


In [7]:
 #Calculate learning rate and radius
decay_function = tf.subtract(1.0, tf.div(iter_ip, num_iter))
current_learning_rate = tf.multiply(learning_rate, decay_function)
current_radius = tf.multiply(radius, decay_function)

In [8]:
bmu_matrix = tf.stack([bmu_location for i in range(x*y)])
bmu_distance = tf.reduce_sum(tf.pow(tf.subtract(locations, bmu_matrix), 2), 1)
neighbourhood_func = tf.exp(tf.negative(tf.div(tf.cast(bmu_distance, "float32"), tf.pow(current_radius, 2))))
learning_rate_matrix = tf.multiply(current_learning_rate, neighbourhood_func)

In [9]:
#Update all the weights
multiplytiplier = tf.stack([tf.tile(tf.slice(
    learning_rate_matrix, np.array([i]), np.array([1])), [input_dim])
                                   for i in range(x*y)])
delta = tf.multiply(
    multiplytiplier,
    tf.subtract(tf.stack([ip for i in range(x*y)]),weights))                

new_weightages = tf.add(weights, delta)
training = tf.assign(weights, new_weightages)                                       

#Initilize session and run it
sess = tf.Session()
initialization = tf.global_variables_initializer()
sess.run(initialization)

In [10]:
from sklearn.datasets import load_iris

In [11]:
data, target = load_iris(True)

In [12]:
data.shape

(150, 4)

In [13]:
target.shape

(150,)

In [14]:
#training
for iter_no in range(num_iter):
    for input_vect in data:
        sess.run(training,
                        feed_dict={ip: input_vect,
                                    iter_ip: iter_no})

centroid_matrix = [[] for i in range(x)]
weights_list = list(sess.run(weights))
locations = list(sess.run(locations))
for i, loc in enumerate(locations):
    centroid_matrix[loc[0]].append(weights_list[i])

In [17]:
#testing
input_vectors= [data[0],data[1]]
return_value = []
for vect in input_vectors:
    min_index = min([i for i in range(len(weights_list))],
                        key=lambda x: np.linalg.norm(vect - weights_list[x]))
    return_value.append(locations[min_index])
print(return_value)

[array([4, 3]), array([4, 4])]


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
# Plotting the response for each pattern in the iris dataset
#plt.pcolor(som.distance_map().T, cmap='bone_r')  # plotting the distance map as background
#plt.colorbar()

t = np.zeros(len(target), dtype=int)
t[target == 0] = 0
t[target == 1] = 1
t[target == 2] = 2

# use different colors and markers for each label
markers = ['o', 's', 'D']
colors = ['C0', 'C1', 'C2']
for cnt, xx in enumerate(data):
    #print(cnt)
    w = som.winner(xx)  # getting the winner
    # place a marker on the winning position for the sample xx
    #print(colors[t[cnt]])
    plt.plot(w[0]+.5, w[1]+.5, markers[t[cnt]], markerfacecolor='None',
             markeredgecolor=colors[t[cnt]], markersize=12, markeredgewidth=2)
plt.axis([0, 10, 0, 10])
plt.savefig('iris.png')
plt.show()